In [ ]:
# ! pip install cdsapi

import cdsapi
import os
import time

: 

In [ ]:
notebook_dir = os.getcwd()
print(notebook_dir)

In [ ]:
os.chdir('/nethome/tliu479/')

In [ ]:
config_content = """{
    url: https://cds-beta.climate.copernicus.eu/api
    key: a18d0e29-dcb1-4197-87ea-1cdbfdf4d9b9
}
"""

# Write the content to .cdsapirc
with open('.cdsapirc', 'w') as config_file:
    config_file.write(config_content)

print("Configuration file created successfully.")


In [ ]:
! ls

In [ ]:
with open('/nethome/tliu479/.cdsapirc', 'r') as file:
    print(file.read())

In [ ]:
os.chdir(notebook_dir)

In [ ]:

# Define the folder path where the GRIB files will be saved
folder_path = '../data/raw/GRIB'

# Ensure the folder exists (create if necessary)
os.makedirs(folder_path, exist_ok=True)


In [ ]:
c = cdsapi.Client()

c.retrieve(
    'reanalysis-era5-land',
    {
        'variable': '2m_temperature',
        'year': '2023',
        'day': '01',
        'month': '01',
        'time': '00:00',
        'area': [
            31.0954, 72.4750, 29.4319, 74.802
        ],
        'format': 'grib',
    },
    '2023.grib')

print('finished')

In [ ]:


# Define the folder path where the GRIB files will be saved
folder_path = '../data/raw/GRIB'

# Ensure the folder exists (create if necessary)
os.makedirs(folder_path, exist_ok=True)

def download_era5_data(start_year, end_year, area, variables):
    """
    Downloads ERA5 data for a specified range of years, each day, and each hour of the day.
    Tracks and prints the time taken for each download.
    
    Parameters:
    - start_year (int): The starting year of the range.
    - end_year (int): The ending year of the range.
    - area (list): The bounding box for the area (North, West, South, East).
    - variables (list): List of variables to download (e.g., 2m_temperature).
    """
    # Loop through each year from start_year to end_year
    for year in range(start_year, end_year + 1):
        # Set the start and end dates for the current year
        start_date = datetime(year, 1, 1)
        end_date = datetime(year, 12, 31)

        # Loop through each day in the current year
        current_date = start_date
        while current_date <= end_date:
            # Extract the year, month, and day
            year_str = current_date.strftime('%Y')
            month_str = current_date.strftime('%m')
            day_str = current_date.strftime('%d')

            # Loop through all hours (00:00 to 23:00)
            for hour in range(24):
                # Format the hour as two digits
                hour_str = f"{hour:02d}:00"

                # Filename for the GRIB file, with the folder path included
                filename = os.path.join(folder_path, f"era5_land_{year_str}{month_str}{day_str}_{hour_str.replace(':', '')}.grib")

                # Initialize the CDS API client
                c = cdsapi.Client()

                # Start the timer
                start_time = time.time()

                # Make the API call to retrieve data for the specific day and hour
                c.retrieve(
                    'reanalysis-era5-land',
                    {
                        'variable': variables,
                        'year': year_str,
                        'month': month_str,
                        'day': day_str,
                        'time': hour_str,  # Request for this specific hour
                        'area': area,
                        'format': 'grib',
                    },
                    filename
                )

                # End the timer
                end_time = time.time()

                # Calculate the time taken
                download_time = end_time - start_time

                # Print a message when the file is saved, including the time taken
                print(f"Downloaded data for {year_str}-{month_str}-{day_str} {hour_str} and saved as {filename}")
                print(f"Time taken for download: {download_time:.2f} seconds")

            # Move to the next day
            current_date += timedelta(days=1)


In [ ]:
# Parameters
area = [31.0954, 72.4750, 12.4319, 94.802]  # North, West, South, East bounding box
variables = ['2m_temperature', '10m_u_component_of_wind', '10m_v_component_of_wind', 'total_precipitation']

# Call the function to download data from 2014 to 2024
download_era5_data(2014, 2014, area, variables)